In [1]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

from selenium.common.exceptions import TimeoutException

import helper

import json
import os
import sys

In [2]:
course_schedule = []
NO_WINDOW = False
IMPLICIT_WAIT_TIME = 20
EXPLICIT_WAIT_TIME = 20
WINDOWS_GRID_X = 3
WINDOWS_GRID_Y = 2
SCREEN_WIDTH = 2560
SCREEN_HEIGHT = 1440

WINDOW_WIDTH = SCREEN_WIDTH / WINDOWS_GRID_X
WINDOW_HEIGHT = SCREEN_HEIGHT / WINDOWS_GRID_Y

RETRY_INTERVAL = 65

In [3]:
def sync_get_element_by_xpath(driver, xpath):
    wait = WebDriverWait(driver, 15)
    try:
        wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element_by_xpath(xpath)

def teacher_norm(teacher):
    if not teacher.startswith('-'):
        return teacher
    tmp = teacher.split('\n')[0].split('- ')[1]
    return tmp[:-2]

def search_courses_by_date(date_str, courses):
    results = []
    
    date_str = date_str.strip()
    
    for item in courses:
        if date_str in item['date']:
            results.append(item)
    return results

def search_courses_by_teacher(teacher_str, courses):
    results = []
    
    teacher_str = teacher_str.strip()
    
    for item in courses:
        if teacher_str in item['teacher']:
            results.append(item)
    return results

def search_courses_by_name(name_str, courses):
    results = []
    
    name_str = name_str.strip()
    
    for item in courses:
        if name_str in item['name']:
            results.append(item)
    return results

def search_courses_by_time(time_str, courses):
    results = []
    
    time_str = time_str.strip()
    
    for item in courses:
        if time_str in item['time']:
            results.append(item)
    return results

    
def list_courses_to_be_booked(courses_to_be_booked):
    cur_date = ''
    if len(courses_to_be_booked) <= 0:
        return
    
    for course in courses_to_be_booked:  

        if cur_date != course['date']:
            if cur_date != '':
                print("-----------------------")
            print("Date: {}".format(course['date']))
            print("-----------------------\n")
            cur_date = course['date']
        print("Name: {}".format(course['name']))
        print("Teacher: {}".format(course['teacher']))
        print("Time: {}".format(course['time']))
        print("\n")
        

In [4]:
def create_new_driver():
    #setup option for chrome profile
    # chrome_options = Options()
    # chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
    options = Options()
    
    if NO_WINDOW:
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')

    #start web driver
    driver = webdriver.Chrome('/Users/huohsien/.wdm/drivers/chromedriver/mac64/114.0.5735.90/chromedriver', options=options)
#     driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.implicitly_wait(IMPLICIT_WAIT_TIME)

    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    return driver, wait

def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)
        
def sign_in(driver, wait):
    
#     username = driver.find_element_by_id('cphContents_txtUsername')
#     password = driver.find_element_by_id('cphContents_txtPassword')
    username = helper.sync_get_element_by_xpath(driver, "//*[@id='ctl00_cphContents_txtUsername']")
    password = helper.sync_get_element_by_xpath(driver, "//*[@id='ctl00_cphContents_txtPassword']")
    if username is not None and password is not None:
        username.send_keys("DN23035242")
        password.send_keys("NTW3BSFH")


        wait.until(ec.element_to_be_clickable((By.CLASS_NAME, "submit-button")))
    #     wait.until(ec.presence_of_element_located((By.XPATH, "//div[@class='submit-button']/input")))

        sign_in_btn = driver.find_element_by_class_name("submit-button") 
        sign_in_btn.click()      
        

def log_out(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
    
def click_book_for_class(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'Book for Class')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()

def click_book_this_class_now(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'BOOK THIS CLASS NOW')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
        return True
    else:
        return False

In [5]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [6]:
temp = []

# ---------------

with open(r'./class_schedules/17 July - 23 July 2023.json', "r") as read_file:
    course_schedule = json.load(read_file)
# ---------------


Day_string = "22"

c = helper.search_courses_by_date(Day_string, course_schedule)
c = helper.search_courses_by_name('Flow', c)
c = helper.search_courses_by_teacher('Moor', c)
# c = helper.search_courses_by_time('PM', c)
temp.extend(c)

courses_to_be_booked = temp

list_courses_to_be_booked(courses_to_be_booked)

Date: 22 Jul Sat
-----------------------

Name: Flow Yoga 動瑜珈
Teacher: Moorthi
Time: 11:00 - 12:00 PM




In [7]:
master_counter = 1

In [8]:
def sign_in_once(driver, wait):
#     print("sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    sign_in(driver, wait)

In [9]:
def timestamp_string():

    date_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
#     date_time = datetime.datetime.now().strftime("%S")

    return date_time

In [10]:
csv_file_name = "./log_try_booking_a_class_that_was_full-- {}.csv".format(timestamp_string())

In [11]:
class_idx = 1
num_classes = len(courses_to_be_booked)

In [12]:
def reserve_class(driver, wait, course_to_be_booked):
    
    global master_counter
    global class_idx
    global num_classes

    stride =RETRY_INTERVAL / 60 * 15 * num_classes

    book_url = course_to_be_booked['href']
    class_name = course_to_be_booked['name']
    teacher = course_to_be_booked['teacher']
    
#     reserve_class_log_dict = {}
    
#     reserve_class_log_dict['name'] = class_name
    driver.get(book_url)
    
    if click_book_this_class_now(driver, wait):
        
        print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        log_str = "'{}','{}' by '{}',BOOKED,'{}'\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        print(log_str)
        
        with open(csv_file_name, "a") as write_file:
            write_file.write(log_str)
        num_classes = num_classes - 1
    else:
        
#         print("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=RETRY_INTERVAL)
        log_str = "'{}','{}' by '{}',FULL,'{}'\nnext_book_datetime={}".format(master_counter, class_name, teacher, datetime.datetime.now(), next_book_datetime)
        print(log_str)
        
        if 0==0: #(master_counter - class_idx) % stride == 0:
            
            with open(csv_file_name, "a") as write_file:
                write_file.write(log_str)
       
        sign_in_once(driver, wait)
        
        scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked])
        master_counter = master_counter + 1
    
    if class_idx < num_classes:
        class_idx = class_idx + 1
    else:
        class_idx = 1


In [13]:
drivers = []
waits = []
scheduler = BackgroundScheduler()
start = time.time()

#CSV file table header
with open(csv_file_name, "w") as write_file:
            write_file.write("#,Class,Status,Time\n")
        
for idx, course_to_be_booked in enumerate(courses_to_be_booked):

    driver, wait = create_new_driver()
    driver.set_window_position(WINDOW_WIDTH * (idx % 3), WINDOW_HEIGHT * int(idx / 3));
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)

    sign_in_once(driver, wait)
    next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=30+(idx%len(course_to_be_booked))*10 )
    print("now={}".format(datetime.datetime.now()))
    print("next_book_datetime={}".format(next_book_datetime))
    scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked])
    
# print("Time Elapsed: {:.2f}".format(time.time() - start)) 
scheduler.start()

now=2023-07-21 12:38:39.294486
next_book_datetime=2023-07-21 12:39:09.294472
'1','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 12:39:29.985662'
next_book_datetime=2023-07-21 12:40:34.985644
'2','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 12:40:55.710657'
next_book_datetime=2023-07-21 12:42:00.710616
'3','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 12:42:21.301150'
next_book_datetime=2023-07-21 12:43:26.301125
'4','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 12:43:47.001823'
next_book_datetime=2023-07-21 12:44:52.001807
'5','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 12:45:12.667190'
next_book_datetime=2023-07-21 12:46:17.667175
'6','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 12:46:38.284808'
next_book_datetime=2023-07-21 12:47:43.284792
'7','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 12:48:03.948268'
next_book_datetime=2023-07-21 12:49:08.948241
'8','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 12:49:29.700606'
next_book_datetime=2023-07-21 12:50:34.700571
'9','Flow Yoga 動瑜珈'

'73','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 14:22:17.966755'
next_book_datetime=2023-07-21 14:23:22.966736
'74','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 14:23:43.559247'
next_book_datetime=2023-07-21 14:24:48.559232
'75','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 14:25:09.455395'
next_book_datetime=2023-07-21 14:26:14.455374
'76','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 14:26:35.119416'
next_book_datetime=2023-07-21 14:27:40.119390
'77','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 14:28:00.753820'
next_book_datetime=2023-07-21 14:29:05.753807
'78','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 14:29:26.408329'
next_book_datetime=2023-07-21 14:30:31.408307
'79','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 14:30:52.043048'
next_book_datetime=2023-07-21 14:31:57.043030
'80','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 14:32:17.748176'
next_book_datetime=2023-07-21 14:33:22.748148
'81','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 14:33:43.355611'
next_book_datetime=2

'145','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 16:05:07.084976'
next_book_datetime=2023-07-21 16:06:12.084960
'146','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 16:06:32.768520'
next_book_datetime=2023-07-21 16:07:37.768503
'147','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 16:07:58.420907'
next_book_datetime=2023-07-21 16:09:03.420890
'148','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 16:09:24.046829'
next_book_datetime=2023-07-21 16:10:29.046813
'149','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 16:10:49.676434'
next_book_datetime=2023-07-21 16:11:54.676410
'150','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 16:12:15.321474'
next_book_datetime=2023-07-21 16:13:20.321454
'151','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 16:13:40.956501'
next_book_datetime=2023-07-21 16:14:45.956484
'152','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 16:15:06.562970'
next_book_datetime=2023-07-21 16:16:11.562945
'153','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 16:16:32.208348'
next_book_d

'217','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 17:47:55.344837'
next_book_datetime=2023-07-21 17:49:00.344819
'218','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 17:49:21.168835'
next_book_datetime=2023-07-21 17:50:26.168824
'219','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 17:50:46.794033'
next_book_datetime=2023-07-21 17:51:51.794016
'220','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 17:52:12.515253'
next_book_datetime=2023-07-21 17:53:17.515235
'221','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 17:53:38.142176'
next_book_datetime=2023-07-21 17:54:43.142151
'222','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 17:55:03.754051'
next_book_datetime=2023-07-21 17:56:08.754039
'223','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 17:56:29.462752'
next_book_datetime=2023-07-21 17:57:34.462727
'224','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 17:57:55.116747'
next_book_datetime=2023-07-21 17:59:00.116730
'225','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 17:59:20.766918'
next_book_d

'289','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 19:30:43.468167'
next_book_datetime=2023-07-21 19:31:48.468142
'290','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 19:32:09.078398'
next_book_datetime=2023-07-21 19:33:14.078382
'291','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 19:33:34.679278'
next_book_datetime=2023-07-21 19:34:39.679252
'292','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 19:35:00.301667'
next_book_datetime=2023-07-21 19:36:05.301654
'293','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 19:36:25.912062'
next_book_datetime=2023-07-21 19:37:30.912051
'294','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 19:37:51.539594'
next_book_datetime=2023-07-21 19:38:56.539576
'295','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 19:39:17.207780'
next_book_datetime=2023-07-21 19:40:22.207767
'296','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 19:40:42.832584'
next_book_datetime=2023-07-21 19:41:47.832547
'297','Flow Yoga 動瑜珈' by 'Moorthi',FULL,'2023-07-21 19:42:08.427872'
next_book_d